Energy consumption for housholds. Data is published on EIA website, reflects Energy Consumption Data from National survey.


Original data in given in **recs2015_public_v4.csv**, each column is a SAS variable for houshold parameters, consumption etc.
Each row is a sample of houshold with given parameters. **NWEIGHT** column - amount of households this sample represents.
There is also **codebook_publicv4.CSV** file with descriptions of SAS vareables

See more here:
https://www.eia.gov/consumption/residential/reports.php

<table>
    <tr>
        <th>
            <img src="Pics_for_notebook/Codebook_view.jpg">
        </th>
        <th>
            <img src="Pics_for_notebook/recs_view.jpg">
        </th>
    </tr>
</table>


In [1]:
import numpy as np
import os
import pandas as pd
import pickle

In [2]:
#getting data
path = os.getcwd()
pdData = pd.read_csv(os.path.join(path,'recs2015_public_v4.csv'), delimiter=',')
pdData = pdData.dropna().reset_index()
m_s, n_s = pdData.shape # m_s - number of samples, n_s - number of parameters+levels for each sample

I devided **codebook** on two separate codebooks: for **Features** - parameters of household and **Labels** - what we may want to predict
Constants and Z columns (impured/not imputed, see https://www.eia.gov/consumption/residential/reports.php) were cleaned from codebooks. Thus, **codebook_edited_shortened.xlsx** - codebooks with parameters of household used for models and **codebook_for_Labels_all.xlsx** - codebooks with Labels
Cath# column is a technical column used for GUI and corresponding to the Tab number
<table>
    <tr>
        <th>
            <img src="Pics_for_notebook/New_Codebook_view.jpg">
        </th>
        <th>
            <img src="Pics_for_notebook/New_Lab_Codebook_view.jpg">
        </th>
    </tr>
</table>

In [3]:
#Making Labels_list and Y
pdCdbkLabels = pd.read_excel(os.path.join(path,'codebook_for_Labels_all.xlsx'), delimiter=',', header=0)
Labels_list = pdCdbkLabels['SAS Variable Name'].to_list() 
Y = pdData[Labels_list].copy()

In [4]:
#Making SAS_var_list
pdCodebook = pd.read_excel(os.path.join(path,'codebook_edited_shortened.xlsx'), delimiter=',', header=0)
SAS_var_list = pdCodebook['SAS Variable Name'].to_list()

Most of the columns are integers (Type: Numeric in codebook), but some are coded with letters (Type: Character in codebook). To use the data for ML, we need them all uniform - integer. The function **code** codes character cells to integer.

In [5]:
def code(key):
    Char_data_dict = {
        'METRO':0,
        'MICRO':1,
        'NONE':2,        
        'U':0,
        'C':2,
        'R':3,     
        '1A-2A':1,
        '2B':2,
        '3A':3,
        '3B-4B':4,
        '3C':5,
        '4A':6,
        '4C':7,
        '5A':8,
        '5B-5C':9,
        '6A-6B':10,
        '7A-7B-7AK-8AK':11,
        'Cold/Very Cold': 1,
        'Hot-Dry/Mixed-Dry':2,
        'Hot-Humid':3,
        'Mixed-Humid':4,
        'Marine':5
    }
    try: ret=Char_data_dict[key] 
    except: ret=key #if it's already int in some reason
    return ret


In [6]:
# Code all charactel columns into integer 
Charact_list =pdCodebook.where(pdCodebook['Type']=='Character')['SAS Variable Name'].dropna().to_list()
for col in Charact_list:
    pdData[col] = pdData[col].apply(code)    

In [7]:
#Making X, after everithng int
X = pdData[SAS_var_list].copy()

Original data is wrapped to "samples", representing households with similar parameters. NWEIGHT reflects how many of these housholds are wrapped to the sample.
To use the data for model training and validation, we need "unwrap" it - simulate original dataset from survey. **X_extruded and Y_extruded - rows of data repeated ** NWEIGHT/min(NWEIGHT)** in numpy array. 
**pd_x and pd_y** - pandas dataframes of them

In [8]:
#NWEIGHT
NWEIGHT=pdData['NWEIGHT'].copy()
NWEIGHT = NWEIGHT/np.min(NWEIGHT)
NWEIGHT = NWEIGHT.apply(round) #how many times row should be repeated

In [9]:
#This code takes a while to run, unwrapping data into simulated datasat with given  NWEIGHT repeats.
m_s, n_s = X.shape
X_extruded = np.empty([0, n_s])  #numpy array
for s in range(m_s):
    for r in range(NWEIGHT[s]):  #row by row
        tmp = np.array(X.loc[s]).reshape(1,n_s)
        X_extruded =np.vstack([X_extruded, tmp])
pd_x  = pd.DataFrame(data = X_extruded,columns = X.columns.values)

m_s, n_s_y = Y.shape
Y_extruded = np.empty([0, n_s_y])  #numpy array
for s in range(m_s):
    for r in range(NWEIGHT[s]):  #row by row
        tmp = np.array(Y.loc[s]).reshape(1,n_s_y)
        Y_extruded = np.vstack([Y_extruded, tmp])
pd_y  = pd.DataFrame(data = Y_extruded, columns = Y.columns.values)

Data_to_pickle = {
    'pd_x': pd_x, 'pd_y': pd_y, 'pdCodebook':pdCodebook
}

dbfile = open('pd_x_pd_y_pdCodebook', 'wb') 
pickle.dump(Data_to_pickle, dbfile)                      
dbfile.close() 

In [10]:
#Saved_Features
#Optional, needed just to play with GUI. 
#It saves 5 most common household samples into scv file compatible with GUI
for indx in range(5):
    SampleData = pd.DataFrame(X.loc[indx]).transpose()
    SampleData.to_csv('SavedFeatures//' +str(indx)+'.csv', index=False)